In [ ]:
from afinn import Afinn
from collections import defaultdict

# Load AFINN lexicon with defaultdict to handle missing words
sent_lexicon = defaultdict(lambda: 0, Afinn()._dict)

# Function to calculate sentiment score for a single text
def calculate_sentiment(text):

    #print(*(sent_lexicon[word] for word in text), sep='\n')
    
    # Sum up the sentiment scores of each word in the text
    sentiment_score = sum(sent_lexicon[word] for word in text)
    
    # Optionally normalize by the number of words (average sentiment per word)
    if len(text) > 0:
        normalized_score = sentiment_score / len(text)
    else:
        normalized_score = 0

    # if normalized_score == 0:
    #     print('text:', text)
    #     print('sentiment_score:', sentiment_score)
    #     print('len(text):', len(text))
    #     print('normalized_score:', normalized_score)
    # else:
    #     print('text:', text)
    #     print('sentiment_score:', sentiment_score)
    #     print('len(text):', len(text))
    #     print('normalized_score:', normalized_score)
    if normalized_score != 0:
        print('text:', text)
        print('sentiment_score:', sentiment_score)
        print('len(text):', len(text))
        print('normalized_score:', normalized_score)
        
    return normalized_score